In [4]:
%run GpuOptions.ipynb
%run BuildTrainingData.ipynb
%run TrendTools.ipynb

Found 6085 images belonging to 4 classes.
Found 1520 images belonging to 4 classes.


In [5]:
import os
import tensorflow as tf
from tensorflow.keras.layers import *
import common.model as m

classes = training_data_generator.class_indices
data, _ = training_data_generator.next()
shape = data[0].shape
inputs = tf.keras.Input(shape=shape)

outputs = m.chained(
    # Entry Flow
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    m.expand(
        flow=lambda previous_activation, size: tf.keras.layers.add([
            m.chained(
                m.duplicate(
                    layers=lambda: [
                        tf.keras.layers.Activation('relu'),
                        tf.keras.layers.SeparableConv2D(filters=size, kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                    ],
                    count=2
                ),
                tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
            )(previous_activation),
            tf.keras.layers.Conv2D(filters=size, kernel_size=1, strides=2, padding='same')(previous_activation),
        ]),
        values=[128, 256, 728],
    ),

    # # Middle Flow
    m.expand(
        flow=lambda previous_activation, _: tf.keras.layers.add([
            m.duplicate(
                layers=lambda: [
                    tf.keras.layers.Activation('relu'),
                    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                ],
                count=3,
            )(previous_activation),
            previous_activation,
        ]),
        values=[0] * 8
    ),

    # # Exit Flow
    lambda previous_activation: tf.keras.layers.add([
        m.chained(
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
        )(previous_activation),
        tf.keras.layers.Conv2D(filters=1024, kernel_size=1, strides=2, padding='same')(previous_activation),
    ]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes), activation='linear'),
    tf.keras.layers.Dropout(0.2),
)(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.build(input_shape=(None, *shape))
model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.008),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'],
)
if os.path.exists('isthemountainout.best.h5'):
    model.load_weights('isthemountainout.best.h5')
elif os.path.exists('isthemountainout.h5'):
    model.load_weights('isthemountainout.h5')
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 112, 112, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 112, 112, 32) 128         conv2d_6[0][0]                   
__________________________________________________________________________________________________
activation_36 (Activation)      (None, 112, 112, 32) 0           batch_normalization_36[0][0]     
_______________________________________________________________________________________

In [6]:
from datetime import datetime

model.fit(
    training_data_generator,
    epochs=700,
    verbose=True,
    steps_per_epoch=training_data_generator.samples // 
        training_data_generator.batch_size,
    validation_data=validation_data_generator,
    validation_steps=validation_data_generator.samples // 
        validation_data_generator.batch_size,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'isthemountainout.best.h5',
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            verbose=True),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=50,
            restore_best_weights=True,
            verbose=True),
        # tf.keras.callbacks.CSVLogger(os.path.join('logs', 'isthemountainout.training.csv')),
        tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join('logs', 'fit', datetime.now().strftime('%Y%m%d%H%M%S')),
            update_freq=50,
            write_images=True,
            write_graph=True,
            embeddings_freq=10),
        m.LogConfusionMatrixCallback(
            model=model,
            datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)\
                .flow_from_directory(data_directory, batch_size=3096, shuffle=True, target_size=image_size),
            logdir=os.path.join('logs', 'image'))
    ])

Found 7605 images belonging to 4 classes.
Epoch 1/700
1521/1521 [==============================] - ETA: 0s - loss: 0.6525 - accuracy: 0.8382
Epoch 00001: val_loss improved from inf to 0.54704, saving model to isthemountainout.best.h5
1521/1521 [==============================] - 216s 142ms/step - loss: 0.6525 - accuracy: 0.8382 - val_loss: 0.5470 - val_accuracy: 0.9039
Epoch 2/700
1521/1521 [==============================] - ETA: 0s - loss: 0.6233 - accuracy: 0.8551
Epoch 00002: val_loss improved from 0.54704 to 0.53147, saving model to isthemountainout.best.h5
1521/1521 [==============================] - 201s 132ms/step - loss: 0.6233 - accuracy: 0.8551 - val_loss: 0.5315 - val_accuracy: 0.9125
Epoch 3/700
1521/1521 [==============================] - ETA: 0s - loss: 0.6094 - accuracy: 0.8564
Epoch 00003: val_loss improved from 0.53147 to 0.52893, saving model to isthemountainout.best.h5
1521/1521 [==============================] - 187s 123ms/step - loss: 0.6094 - accuracy: 0.8564 - val

Epoch 28/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5324 - accuracy: 0.9043
Epoch 00028: val_loss did not improve from 0.52717
1521/1521 [==============================] - 168s 110ms/step - loss: 0.5324 - accuracy: 0.9043 - val_loss: 0.5602 - val_accuracy: 0.9033
Epoch 29/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5539 - accuracy: 0.8878
Epoch 00029: val_loss did not improve from 0.52717
1521/1521 [==============================] - 168s 111ms/step - loss: 0.5539 - accuracy: 0.8878 - val_loss: 0.5734 - val_accuracy: 0.9026
Epoch 30/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5413 - accuracy: 0.8992
Epoch 00030: val_loss did not improve from 0.52717
1521/1521 [==============================] - 160s 105ms/step - loss: 0.5413 - accuracy: 0.8992 - val_loss: 0.5792 - val_accuracy: 0.9053
Epoch 31/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5298 - accuracy: 0.9046
Epoch 00031: val_loss did not improv

1521/1521 [==============================] - ETA: 0s - loss: 0.5206 - accuracy: 0.9061
Epoch 00056: val_loss did not improve from 0.52717
1521/1521 [==============================] - 168s 111ms/step - loss: 0.5206 - accuracy: 0.9061 - val_loss: 0.5598 - val_accuracy: 0.9145
Epoch 57/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5246 - accuracy: 0.9064
Epoch 00057: val_loss did not improve from 0.52717
1521/1521 [==============================] - 160s 105ms/step - loss: 0.5246 - accuracy: 0.9064 - val_loss: 0.5603 - val_accuracy: 0.9112
Epoch 58/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5302 - accuracy: 0.8951
Epoch 00058: val_loss did not improve from 0.52717
1521/1521 [==============================] - 169s 111ms/step - loss: 0.5302 - accuracy: 0.8951 - val_loss: 0.5519 - val_accuracy: 0.9112
Epoch 59/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5222 - accuracy: 0.9081
Epoch 00059: val_loss did not improve from 0.5271